In [71]:
import gensim
from gensim import corpora
import os
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import warnings
warnings.filterwarnings('ignore')  # To ignore all warnings that arise here to enhance clarity
from gensim.models.coherencemodel import CoherenceModel
from gensim.models.ldamodel import LdaModel
!pip install pyLDAvis
import jieba
import pandas as pd
from gensim.models import LdaModel
from gensim.corpora import Dictionary
import matplotlib.pyplot as plt
import pyLDAvis.gensim_models as pg
import matplotlib

ImportError: cannot import name 'corpora' from 'gensim' (unknown location)

In [ ]:
data = pd.read_csv(""/Desktop/Spring 2024/POLI179/poli 179 data/People's daily/Train/final_data.csv")

In [ ]:
stopwords_files = ['baidu_stopwords.txt', 'cn_stopwords.txt', 
                   'hit_stopwords.txt', 'scu_stopwords.txt']
stopwords = set()
for filename in stopwords_files:
    file_path = os.path.join('/content/drive/My Drive/POLI/POLI179_final_project/Stopwords/', filename)
    with open(file_path, 'r', encoding='utf-8-sig') as file:
        stopwords.update([line.strip() for line in file.readlines()])
        
additional_stopwords = {'年', '月', '日', '第版', '说', '中', '更','年 月 日','月 日','时','我国','杨丽','总','这是','记者','做','里','一年','包括','年月日','一是',
                        '二是','要','新','亿元','没','岁','想', '应','一种','郑', '走','老', '前','干','吃','越来越','记者','进一步','元','来到','找','事','带','买',
                        '成','住','本版','本报记者','达','占','镇','村','名','区','来到','一条','建成','性','绣','黄花','去年','提出','介绍','出','搬','李心萍','万元',
                        '亿元','万人','千人','周永康','嫌疑人','第一节','第二节','第三节','万亿元','加快','支农','出','水','据','一季度','二季度',
                        '三季度','四季度','上年','比上','拉动','',''}  
stopwords.update(additional_stopwords)

print("Total stopwords loaded:", len(stopwords))

In [ ]:
def preprocess_text(text):
    words = jieba.cut(text)
    filtered_words = [word for word in words if word not in stopwords]
    return filtered_words
data['segmented_text'] = data['TextBody'].apply(preprocess_text)
dictionary = Dictionary(data['segmented_text'])
corpus = [dictionary.doc2bow(text) for text in data['segmented_text']]

In [ ]:
lda = LdaModel(corpus=corpus, num_topics=5, id2word=dictionary, passes=30, random_state=1)

In [ ]:
#计算困惑度
def perplexity(num_topics):
    ldamodel = LdaModel(corpus, num_topics=num_topics, id2word = dictionary, passes=30)
    print(ldamodel.print_topics(num_topics=num_topics, num_words=15))
    print(ldamodel.log_perplexity(corpus))
    return ldamodel.log_perplexity(corpus)
#计算coherence
def coherence(num_topics):
    ldamodel = LdaModel(corpus, num_topics=num_topics, id2word = dictionary, passes=30,random_state = 1)
    print(ldamodel.print_topics(num_topics=num_topics, num_words=10))
    ldacm = CoherenceModel(model=ldamodel, texts=data['segmented_text'], dictionary=dictionary, coherence='u_mass')
    print(ldacm.get_coherence())
    return ldacm.get_coherence()

In [ ]:
x = range(1,15)
y = [coherence(i) for i in x]
plt.plot(x, y)
plt.xlabel('Number of Topics')
plt.ylabel('coherence')
plt.rcParams['font.sans-serif']=['SimHei']
matplotlib.rcParams['axes.unicode_minus']=False
plt.title('Topic-Coherence Change Trend')
plt.show()

In [ ]:
x = range(1,15)
z = [perplexity(i) for i in x]
plt.plot(x, y)
plt.xlabel('Number of Topics')
plt.ylabel('Perplexity')
plt.rcParams['font.sans-serif']=['SimHei']
matplotlib.rcParams['axes.unicode_minus']=False
plt.title('Topic-Perplexity Change Trend')
plt.show()

In [ ]:
import pyLDAvis

In [ ]:
# Visualize Topics
vis_data = pg.prepare(lda, corpus, dictionary)
pyLDAvis.save_html(vis_data, '/Plot/lda_visualization.html')